In [1]:
spark.sql("show databases").show()

Calculation started (calculation_id=66c72fbc-8ec3-6f24-18a2-671cf7b7f043) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+
|           namespace|
+--------------------+
|athena_spark_tuto...|
|             default|
|                taxi|
|  term_paper_demo_db|
+--------------------+



In [2]:
# Read the data and create df
sqlDF = spark.sql("SELECT * From term_paper_demo_db.nyc_taxi")

Calculation started (calculation_id=30c72fbc-e9c8-2d51-edf4-584493b77a4f) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [3]:
sqlDF.show()

Calculation started (calculation_id=54c72fbc-fe7a-b07d-ebfa-ecf3b3c37461) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|           medallion|        hack_license|    pickup_datetime|   dropoff_datetime|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+--------------------+--------------------+-------------------+-------------------+-----------------+-------------+----------------+---------------+-----------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|5EE2C4D3BF57BDB45...|E96EF8F6E6122591F...|2013-01-01 00:00:09|2013-01-01 00:00:36|               26|          0.1|       -73.99221|      40.725124|       -73.991646|       40.726658|  

In [4]:
sqlDF.createOrReplaceTempView("sqlDF_view")

Calculation started (calculation_id=ccc72fbd-1a95-9f95-59c7-1420f8eed63a) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [5]:
#Remove NAs and 0s 
sqlDF_filtered = spark.sql("""
SELECT *
FROM sqlDF_view
WHERE 
    trip_time_in_secs IS NOT NULL AND CAST(trip_time_in_secs AS FLOAT) != 0 AND
    trip_distance IS NOT NULL AND CAST(trip_distance AS FLOAT) != 0 AND
    fare_amount IS NOT NULL AND CAST(fare_amount AS FLOAT)!= 0 AND
    surcharge IS NOT NULL AND CAST(surcharge AS FLOAT) != 0 AND
    total_amount IS NOT NULL AND CAST(total_amount AS FLOAT) != 0
""")

Calculation started (calculation_id=66c72fbd-3576-b774-0870-7a56d4744647) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [6]:
from pyspark.sql import functions as F

# Group by 'medallion', count distinct 'hack_license'
driver_counts = sqlDF_filtered.groupBy('medallion').agg(F.countDistinct('hack_license').alias('distinct_driver_count'))

# Order by the count in descending order and take the top 10
top_10_medallions = driver_counts.orderBy(F.desc('distinct_driver_count')).limit(10)
top_10_medallions.show()

Calculation started (calculation_id=5ac72fbd-4d96-a584-2bf1-b083310f4a87) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+--------------------+---------------------+
|           medallion|distinct_driver_count|
+--------------------+---------------------+
|7DA8DF1E4414F81EB...|                  305|
|B07944BF31699A169...|                  303|
|6C1132EF70BC0A7DB...|                  300|
|23DB792D3F7EBA030...|                  300|
|A10A65AFD9F401BF3...|                  299|
|E4ED88FCD99B05ED6...|                  298|
|021F88711374FB07E...|                  293|
|7D93E7FC4A7E4615A...|                  292|
|43D85E4D101135DDF...|                  292|
|EE06BD8A621CAC3B6...|                  289|
+--------------------+---------------------+



In [12]:
output_df_1 = top_10_medallions.select(F.concat_ws(", ", "medallion", F.col("distinct_driver_count").cast("string")).alias("text"))

Calculation started (calculation_id=3cc72fc1-81ea-5b0f-bd79-a8b750e07ec2) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [13]:
output_df_1.take(10)

Calculation started (calculation_id=62c72fc1-92bf-d700-28bc-01263654f465) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
[Row(text='7DA8DF1E4414F81EBD3A0140073B2630, 305'), Row(text='B07944BF31699A169091D2B16597A4A9, 303'), Row(text='6C1132EF70BC0A7DB02174592F9A64A1, 300'), Row(text='23DB792D3F7EBA03004E470B684F2738, 300'), Row(text='A10A65AFD9F401BF3BDB79C84D3549E7, 299'), Row(text='E4ED88FCD99B05ED6F072D9A104402A8, 298'), Row(text='021F88711374FB07E02F03A06637C5C8, 293'), Row(text='43D85E4D101135DDFC1BC16DF53665FE, 292'), Row(text='7D93E7FC4A7E4615A34B8286D92FF57F, 292'), Row(text='EE06BD8A621CAC3B608ACFDF0585A76A, 289')]



In [16]:
# Save the DataFrame as a text file in S3
output_df_1.write.text("s3://term-paper-demo/results/output_1_large")

Calculation started (calculation_id=98c72fc3-5820-d10b-ed2c-58ae8739161a) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [17]:
# Count the number of records by the length of trip time
duration_categories = sqlDF_filtered.withColumn(
    'duration_category',
    F.when(F.col('trip_time_in_secs') < 600, 'Less than 10 mins')
     .when((F.col('trip_time_in_secs') >= 600) & (F.col('trip_time_in_secs') < 1200), '10 to 20 mins')
     .when((F.col('trip_time_in_secs') >= 1200) & (F.col('trip_time_in_secs') < 1800), '20 to 30 mins')
     .otherwise('More than 30 mins')
)

trip_counts_by_duration = duration_categories.groupBy('duration_category').count().orderBy('duration_category')
trip_counts_by_duration.show()

Calculation started (calculation_id=40c72fc4-007d-b832-cab0-7407e95c9324) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+-----------------+--------+
|duration_category|   count|
+-----------------+--------+
|    10 to 20 mins|31670435|
|    20 to 30 mins| 8725616|
|Less than 10 mins|41769539|
|More than 30 mins| 2800865|
+-----------------+--------+



In [18]:
output_df_2 = trip_counts_by_duration.select(F.concat_ws(", ", "duration_category", F.col("count").cast("string")).alias("text"))

Calculation started (calculation_id=52c72fc7-d8ed-82b8-5188-c27aa32d3d23) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [19]:
output_df_2.take(10)

Calculation started (calculation_id=e4c72fc7-f048-e3ac-a812-0172f7ba1597) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
[Row(text='10 to 20 mins, 31670435'), Row(text='20 to 30 mins, 8725616'), Row(text='Less than 10 mins, 41769539'), Row(text='More than 30 mins, 2800865')]



In [20]:
# Save the DataFrame as a text file in S3
output_df_2.write.text("s3://term-paper-demo/results/output_2_large")

Calculation started (calculation_id=c2c72fc8-a208-d08f-4db2-a7903af1b75a) in (session=7ac72fbc-744a-3c14-5b2c-314bae970498). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
